In [4]:
import os
import argparse
import torch
import numpy as np
from torch.nn.utils import rnn
from torch.distributions import Bernoulli
from modules import graph_rnn
from utils.data import WaddleDataset
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

dataset = WaddleDataset("data/preprocessed_data/permute.pkl")
num_feat = dataset.discrete_feature_dim + dataset.continuous_feature_dim \
        + dataset.max_vertex_num


In [5]:
run = 1
checkpoint_path = "wadld/outputs/multi_run/run"+ str(run) + "/last.checkpoint"
checkpoint = torch.load(checkpoint_path)

if 'args' in checkpoint.keys():
    rnn_hidden_size = checkpoint['args']['hidden_size']
    rnn_num_layers = checkpoint['args']['num_layers']
else:
    rnn_hidden_size = 64
    rnn_num_layers = 3

model = graph_rnn.GraphRNN(
    discrete_feature_dim=dataset.discrete_feature_dim,
    continuous_feature_dim=dataset.continuous_feature_dim,
    max_vertex_num=dataset.max_vertex_num,
    rnn_hidden_size=rnn_hidden_size,
    rnn_num_layers=rnn_num_layers,
)
model.load_state_dict(checkpoint['model'])
model.eval()


GraphRNN(
  (theta_net): LSTM(385, 64, num_layers=3, batch_first=True)
  (discrete_feature_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (continuous_feature_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=2, bias=True)
  )
  (adjacency_feature_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=382, bias=True)
  )
)

In [22]:
model.continuous_feature_net[2].bias

Parameter containing:
tensor([0.3573, 0.3573], requires_grad=True)